In [2]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

##### Dependencies
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install sentencepiece

In [1]:
!pip install transformers
!pip3 install torch torchvision torchaudio
!pip install sentencepiece

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.0.12 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [3]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the CPU


In [5]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small',
                                             return_dict=True)
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [7]:
train_df=pd.read_csv('dataframe_slice.csv', error_bad_lines=False,engine='python',index_col=0)

train_df=train_df.iloc[  :1000,:]
train_df=train_df.sample(frac = 1)

batch_size=2
num_of_batches=len(train_df)//batch_size

Skipping line 79790: field larger than field limit (131072)
Skipping line 163735: unexpected end of data


In [8]:
train_df

,title,description,title_len,desc_len
609,Telegu Bible,['Largest member of the Dravidian language fam...,12,100
717,"Mighty Bright 84312 2-LED USB Light, Silver",['Squinting at your finger placement in low li...,43,692
42,Microsoft MultiMedia Keyboard 1.0A KB-0168,['Microsoft MultiMedia Keyboard 1.0A KB-0168'],42,46
733,Power Vertical Battery Grip holder as MB-D14 f...,['This is a brand new Battery Grip For Nikon D...,76,391
19,Scared Money,['Edgar-finalist Hime\'s overly complicated se...,12,2393
...,...,...,...,...
636,Barnes &amp; Noble NOOK AC Adapter and USB Cab...,['Keep your NOOK charged and ready for use wit...,52,84
1046,Professional Kobo Glo HD Lossless Data Chargin...,['Professional Kobo Glo HD Lossless Data Charg...,164,160
738,The Legend of the Starfish,['This beautifully illustrated children\'s boo...,26,481
1067,Professional Ultra SanDisk 64GB MicroSDXC Sams...,"['', '', 'This Factory Direct product is from ...",186,189


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 609 to 174
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1000 non-null   object
 1   description  1000 non-null   object
 2   title_len    1000 non-null   int64 
 3   desc_len     1000 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 39.1+ KB


In [9]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [10]:
model.train()
num_of_epochs=1
loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            inputs = row['title']
            labels = row['description']
            inputbatch.append(inputs)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=100,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=100,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
        optimizer.zero_grad()

    # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
        loss.backward()

    #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))

Running epoch: 1


KeyboardInterrupt: 

In [ ]:
tokenizer.